In [ ]:
#get Dataset
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-11-30 10:59:57--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  3.09MB/s    in 71s     

2020-11-30 11:01:09 (4.35 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [ ]:
#Import Lib
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile,os
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
#Create Directory
zip_dir = '/tmp/rockpaperscissors.zip'
extract = zipfile.ZipFile(zip_dir, 'r')
extract.extractall('/tmp')
extract.close()

In [ ]:
data_dir = '/tmp/rockpaperscissors/rps-cv-images'
rock_dir = os.path.join(data_dir, 'rock')
paper_dir = os.path.join(data_dir, 'paper')
scissors_dir = os.path.join(data_dir, 'scissors')

In [ ]:
os.listdir(data_dir)

['README_rpc-cv-images.txt', 'scissors', 'rock', 'paper']

In [ ]:
#Length Dataset
print('Data Paper :',len(os.listdir(paper_dir)))
print('Data Rock :',len(os.listdir(rock_dir)))
print('Data Scissors :',len(os.listdir(scissors_dir)))

Data Paper : 712
Data Rock : 726
Data Scissors : 750


In [ ]:
data1 = len(os.listdir(paper_dir))
data2 = len(os.listdir(rock_dir))
data3 = len(os.listdir(scissors_dir))
jumlah = data1 + data2 + data3
print('Total Sampel Gambar :',jumlah) #Sum total dataset

Total Sampel Gambar : 2188


In [ ]:
dirr = '/tmp/rockpaperscissors/rps-cv-images'
augmen = ImageDataGenerator( #Image Augmentation
    rescale = 1./255,
    horizontal_flip = True,
    height_shift_range = .2,
    vertical_flip = True,
    validation_split = 0.4
)
trg_gen = augmen.flow_from_directory( #Train generator
    dirr,
    target_size = (220,220),
    batch_size = 5,
    class_mode = 'categorical',
    shuffle = True,
    classes = ['rock','paper','scissors'],
    subset = 'training'
)
val_gen = augmen.flow_from_directory( #Validate Generator
    dirr,
    target_size = (220,220),
    batch_size = 5,
    class_mode = 'categorical',
    shuffle = False,
    classes = ['rock','paper','scissors'],
    subset = 'validation'
)

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import MaxPooling2D,Conv2D,Dense,Flatten,Dropout
 #Create Model
model = Sequential([
                    Conv2D(32, (3,3), activation='relu', input_shape=(220, 220, 3)),
                    MaxPooling2D(2,2),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Conv2D(128, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Conv2D(128, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Dropout(0.3),
                    Flatten(),
                    Dense(512, activation='relu'),
                    Dense(3,activation='softmax')
])

In [ ]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', #Compiling Model
              optimizer = RMSprop(),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler
 #Create Callbacks Func
best_model_weights = '/tmp/base.model'
checkpoint = ModelCheckpoint(
    best_model_weights,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto'
)
tensorboard = TensorBoard(
    log_dir = '/tmp/logs',
    histogram_freq=0,
    batch_size=5,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)


reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=40,
    verbose=1, 
    mode='auto',
    cooldown=1 
)

callbacks = [checkpoint,tensorboard,csvlogger,reduce]

In [ ]:
#Train Model
model.fit(
    trg_gen, 
    steps_per_epoch  = 50, 
    validation_data  = val_gen,
    validation_steps = 5,
    epochs = 20, 
    verbose = 1,
    callbacks=callbacks
)

Epoch 1/20
 1/50 [..............................] - ETA: 0s - loss: 1.1063 - accuracy: 0.4000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
50/50 [==============================] - ETA: 0s - loss: 1.3164 - accuracy: 0.3320
Epoch 00001: val_loss improved from inf to 1.07947, saving model to /tmp/base.model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/base.model/assets
50/50 [==============================] - 33s 670ms/step - loss: 1.3164 - accuracy: 0.3320 - val_loss: 1.0795 - val_accuracy: 0.0000e+00
Epoch 2/20
50/50 [================

In [ ]:
%matplotlib inline
upload = files.upload()
#Test Model
for fn in upload.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(220,220))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  hasil = np.argmax(classes)
  print(fn)
  if hasil==0:
    print('paper')
  elif hasil==1:
    print('rock')
  elif hasil==2:
    print('scissors')

NameError: ignored